In [7]:
import pandas as pd
import requests
from config import api_key

In [8]:
# Define the list of meme cryptocurrencies (all above 1 billion MC)
cryptocurrencies = {
    'bitcoin': 'BTC',
    'ethereum': 'ETH',
    'dogecoin': 'DOGE',
    'ripple': 'XRP',
    'chainlink': 'LINK'
}

In [10]:
#Read in tokens

def get_historical_prices(token_id, vs_currency='usd', days=365):
    url = f"https://api.coingecko.com/api/v3/coins/{token_id}/market_chart"
    params = {
        'vs_currency': vs_currency,
        'days': days,
        'interval': 'daily'
    }
    response = requests.get(url, params=params)
    
    if response.status_code != 200:
        print("Response content:", response.content)  # Print the response content for debugging
        raise Exception(f"Failed to fetch data: {response.status_code}")
    
    data = response.json()
    
    # Extracting the price data
    prices = data['prices']
    df = pd.DataFrame(prices, columns=['timestamp', 'price'])
    
    # Convert timestamp to datetime
    df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
    df.set_index('timestamp', inplace=True)
    df.rename(columns={'price': token_id}, inplace=True)
    
    return df

# List to store DataFrames
dfs = []

# Loop through each cryptocurrency and get historical prices
for token_id, symbol in cryptocurrencies.items():
    try:
        df = get_historical_prices(token_id)
        df.rename(columns={token_id: symbol}, inplace=True)
        dfs.append(df)
    except Exception as e:
        print(f"Error fetching data for {token_id}: {e}")

# Concatenate all DataFrames into one, joining on the index (timestamp)
historical_prices_df = pd.concat(dfs, axis=1)

# Display the DataFrame
print(historical_prices_df)


                              BTC          ETH      DOGE       XRP       LINK
timestamp                                                                    
2023-05-23 00:00:00  26869.686022  1818.401464  0.073093  0.462352   6.529927
2023-05-24 00:00:00  27222.937747  1854.193118  0.072956  0.466039   6.504779
2023-05-25 00:00:00  26338.948947  1800.736010  0.070846  0.454365   6.325692
2023-05-26 00:00:00  26475.607903  1805.805760  0.070926  0.454115   6.276259
2023-05-27 00:00:00  26717.987554  1828.956537  0.071095  0.469745   6.351220
...                           ...          ...       ...       ...        ...
2024-05-21 00:40:11           NaN  3678.081886       NaN       NaN        NaN
2024-05-21 00:41:32  71276.581652          NaN       NaN       NaN        NaN
2024-05-21 00:44:43           NaN          NaN       NaN  0.539812        NaN
2024-05-21 00:58:35           NaN          NaN  0.164508       NaN        NaN
2024-05-21 01:21:06           NaN          NaN       NaN       N

In [ ]:
historical_prices_df

,BTC,ETH,DOGE,XRP,LINK
timestamp,,,,,
2023-05-23 00:00:00,26869.686022,1818.401464,0.073093,0.462352,6.529927
2023-05-24 00:00:00,27222.937747,1854.193118,0.072956,0.466039,6.504779
2023-05-25 00:00:00,26338.948947,1800.736010,0.070846,0.454365,6.325692
2023-05-26 00:00:00,26475.607903,1805.805760,0.070926,0.454115,6.276259
2023-05-27 00:00:00,26717.987554,1828.956537,0.071095,0.469745,6.351220
...,...,...,...,...,...
2024-05-21 00:40:11,NaN,3678.081886,NaN,NaN,NaN
2024-05-21 00:41:32,71276.581652,NaN,NaN,NaN,NaN
2024-05-21 00:44:43,NaN,NaN,NaN,0.539812,NaN


In [ ]:
#Keeping up to May 20 2024
historical_prices_df = historical_prices_df.iloc[:-5]
historical_prices_df

,BTC,ETH,DOGE,XRP,LINK
timestamp,,,,,
2023-05-23,26869.686022,1818.401464,0.073093,0.462352,6.529927
2023-05-24,27222.937747,1854.193118,0.072956,0.466039,6.504779
2023-05-25,26338.948947,1800.736010,0.070846,0.454365,6.325692
2023-05-26,26475.607903,1805.805760,0.070926,0.454115,6.276259
2023-05-27,26717.987554,1828.956537,0.071095,0.469745,6.351220
...,...,...,...,...,...
2024-05-16,66220.518314,3035.758955,0.155710,0.518594,13.889619
2024-05-17,65260.879803,2943.585128,0.149711,0.515796,15.500365
2024-05-18,67053.123265,3095.996844,0.155671,0.524203,16.265870


In [ ]:
#Export to csv
historical_prices_df.to_csv('cg_df.csv', index = True)